In [1]:
# use CEASELESS Kernel on adrien local laptop

import os
import xarray as xr
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import cartopy.crs as ccrs # => to add in Seastar env
import seastar as ss
from seastar.utils.tools import dotdict

from scipy import interpolate, stats # => to add in Seastar env ?
from scipy.optimize import least_squares

# from seastar.gmfs.doppler import *

In [2]:
level1 = xr.Dataset(
    data_vars=dict(
            CentralWavenumber=( [],270 ),
            CentralFreq=( [], 13.5 * 10**9 ),
            IncidenceAngleImage=( ['across','along','Antenna'], np.full([5,6,4], 30) ),
            AntennaAzimuthImage=(['across', 'along', 'Antenna'],
                           np.stack((np.full([5, 6], 45),
                                     np.full([5, 6], 90),
                                     np.full([5, 6], 90),
                                     np.full([5, 6], 135)
                                     ), axis=-1
                                    )
                           ),
            Polarization=(['across', 'along','Antenna'],
                          np.stack((np.full([5, 6], 'VV'),
                                    np.full([5, 6], 'VV'),
                                    np.full([5, 6], 'HH'),
                                    np.full([5, 6], 'VV')
                                    ), axis=-1
                                   )
                          ),
#             Sigma0=( ['across','along','Antenna'], np.full([9,11,4], 1.01) ),
#             dsig0=( ['across','along','Antenna'], np.full([9,11,4], 0.05) ),
#             RVL=( ['across','along','Antenna'], np.full([9,11,4], 0.5) ),
#             drvl=( ['across','along','Antenna'], np.full([9,11,4], 0.01) ),
        ),
    coords=dict(
            across=np.arange(0,5),
            along=np.arange(0,6),
            Antenna=['Fore','MidV','MidH','Aft'],
        ),
)
level1 = level1.set_coords([
    'CentralWavenumber',
    'CentralFreq',
    'IncidenceAngleImage',
    'AntennaAzimuthImage',
    'Polarization', 
])


In [3]:
geo = xr.Dataset(
        data_vars=dict(
            WindSpeed=(['across', 'along'], np.full([5, 6], 10)),
            WindDirection=(['across', 'along'], np.full([5, 6], 150)),
            CurrentVelocity=(['across', 'along'], np.full([5, 6], 1)),
            CurrentDirection=(['across', 'along'], np.full([5, 6], 150)),
        ),
        coords=dict(
            across=np.arange(0, 5),
            along=np.arange(0, 6),
        ),
    )
geo

<xarray.Dataset>
Dimensions:           (across: 5, along: 6)
Coordinates:
  * across            (across) int64 0 1 2 3 4
  * along             (along) int64 0 1 2 3 4 5
Data variables:
    WindSpeed         (across, along) int64 10 10 10 10 10 10 ... 10 10 10 10 10
    WindDirection     (across, along) int64 150 150 150 150 ... 150 150 150 150
    CurrentVelocity   (across, along) int64 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    CurrentDirection  (across, along) int64 150 150 150 150 ... 150 150 150 150

In [4]:
level1

<xarray.Dataset>
Dimensions:              (Antenna: 4, across: 5, along: 6)
Coordinates:
    CentralWavenumber    int64 270
    CentralFreq          float64 1.35e+10
    IncidenceAngleImage  (across, along, Antenna) int64 30 30 30 30 ... 30 30 30
    AntennaAzimuthImage  (across, along, Antenna) int64 45 90 90 ... 90 90 135
    Polarization         (across, along, Antenna) <U2 'VV' 'VV' ... 'HH' 'VV'
  * across               (across) int64 0 1 2 3 4
  * along                (along) int64 0 1 2 3 4 5
  * Antenna              (Antenna) <U4 'Fore' 'MidV' 'MidH' 'Aft'
Data variables:
    *empty*

In [5]:
gmf=dotdict({'nrcs': dotdict({'name': 'nscat4ds'})})
gmf['doppler'] = dotdict({'name': 'mouche12'})

In [6]:
level1['Sigma0'] = ss.gmfs.nrcs.compute_nrcs(level1, geo, gmf.nrcs)*1.001

In [7]:
model_rvl_list = [None] * level1.Antenna.size
model_wasv_list = [None] * level1.Antenna.size
for aa, ant in enumerate(level1.Antenna.data):
    model_wasv_list[aa] = ss.gmfs.doppler.compute_wasv(level1.sel(Antenna=ant), geo, gmf=gmf.doppler.name)
    model_rvl_list[aa] = ss.gmfs.doppler.compute_total_surface_motion(level1.sel(Antenna=ant), geo, gmf=gmf.doppler.name)
level1['WASV'] = xr.concat(model_wasv_list, dim='Antenna')*1.001
level1['RVL'] = xr.concat(model_rvl_list, dim='Antenna')*1.001

In [8]:
# Add NaN for RVL for the mid antennas
level1.RVL[1,:,:] = np.full([5,6], np.nan)
level1.RVL[2,:,:] = np.full([5,6], np.nan)


In [9]:
model_rvl_list = [None] * level1.Antenna.size
for aa, ant in enumerate(level1.Antenna.data):
    model_rvl_list[aa] = ss.gmfs.doppler.compute_total_surface_motion(level1.sel(Antenna=ant), geo, gmf='mouche12')

model = level1.drop_vars([var for var in level1.data_vars])
model['RVL'] = xr.concat(model_rvl_list, dim='Antenna')
model['Sigma0'] = ss.gmfs.nrcs.compute_nrcs(level1, geo, gmf=gmf.nrcs)

In [10]:
noise = level1.drop_vars([var for var in level1.data_vars])
# noise = level1.drop_vars(['Sigma0','dsig0','RVL','drvl'])
noise['Sigma0'] = level1.Sigma0*0.05
noise['RVL'] = level1.RVL*0.05

In [11]:
res = (model-level1)/noise

In [12]:
np.concatenate(
    (res.Sigma0.sel(across=1, along=1).data, res.RVL.sel(across=1, along=1).data)
)
    

array([-0.01998002, -0.01998002, -0.01998002, -0.01998002, -0.01998002,
               nan,         nan, -0.01998002])

# Test Fun_residual

In [13]:
sl1 = level1.sel(across=1, along=1)
sn = noise.sel(across=1, along=1)

In [14]:
ss.retrieval.cost_function.fun_residual([5,5,1,1], sl1, sn, gmf)

array([   9.01895349,   12.1681688 ,   12.60884007,   16.59575996,
       -691.66097198,    0.        ,    0.        ,   17.09819452])

### test fun_residual with 2D fields

In [15]:
[geo['U'], geo['V']] = ss.utils.tools.windSpeedDir2UV(geo.WindSpeed, geo.WindDirection)
[geo['C_U'], geo['C_V']] = ss.utils.tools.currentVelDir2UV(geo.CurrentVelocity, geo.CurrentDirection)

In [16]:
geo

<xarray.Dataset>
Dimensions:           (across: 5, along: 6)
Coordinates:
  * across            (across) int64 0 1 2 3 4
  * along             (along) int64 0 1 2 3 4 5
Data variables:
    WindSpeed         (across, along) int64 10 10 10 10 10 10 ... 10 10 10 10 10
    WindDirection     (across, along) int64 150 150 150 150 ... 150 150 150 150
    CurrentVelocity   (across, along) int64 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1
    CurrentDirection  (across, along) int64 150 150 150 150 ... 150 150 150 150
    U                 (across, along) float64 -5.0 -5.0 -5.0 ... -5.0 -5.0 -5.0
    V                 (across, along) float64 8.66 8.66 8.66 ... 8.66 8.66 8.66
    C_U               (across, along) float64 0.5 0.5 0.5 0.5 ... 0.5 0.5 0.5
    C_V               (across, along) float64 -0.866 -0.866 ... -0.866 -0.866

In [17]:
level1

<xarray.Dataset>
Dimensions:              (Antenna: 4, across: 5, along: 6)
Coordinates:
    CentralWavenumber    int64 270
    CentralFreq          float64 1.35e+10
    IncidenceAngleImage  (across, along, Antenna) int64 30 30 30 30 ... 30 30 30
    AntennaAzimuthImage  (across, along, Antenna) int64 45 90 90 ... 90 90 135
    Polarization         (across, along, Antenna) <U2 'VV' 'VV' ... 'HH' 'VV'
  * across               (across) int64 0 1 2 3 4
  * along                (along) int64 0 1 2 3 4 5
  * Antenna              (Antenna) object 'Fore' 'MidV' 'MidH' 'Aft'
Data variables:
    Sigma0               (Antenna, across, along) float64 0.07128 ... 0.1668
    WASV                 (Antenna, across, along) float64 0.3227 ... -1.498
    RVL                  (Antenna, across, along) float64 0.06364 ... -0.5313

In [18]:
results = ss.retrieval.cost_function.fun_residual([geo.U, geo.V, geo.C_U, geo.C_V], level1, noise, gmf)

In [19]:
results

array([[[-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857],
        [-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857],
        [-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857],
        [-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857],
        [-3.04975857, -3.04975857, -3.04975857, -3.04975857,
         -3.04975857, -3.04975857]],

       [[-3.25064737, -3.25064737, -3.25064737, -3.25064737,
         -3.25064737, -3.25064737],
        [-3.25064737, -3.25064737, -3.25064737, -3.25064737,
         -3.25064737, -3.25064737],
        [-3.25064737, -3.25064737, -3.25064737, -3.25064737,
         -3.25064737, -3.25064737],
        [-3.25064737, -3.25064737, -3.25064737, -3.25064737,
         -3.25064737, -3.25064737],
        [-3.25064737, -3.25064737, -3.25064737, -3.25064737,
         -3.25064737, -3.25064737]],

       [[-3.27504276, -3.2

In [20]:
ss.retrieval.cost_function.fun_residual([geo.U, geo.V, geo.C_U, geo.C_V], level1, noise, gmf).shape

(8, 5, 6)

# Test least_squares with fun_residual

In [21]:
opt = {
        'method': 'trf', # Trust Region Reflective algorithm, particularly suitable for large sparse problems with bounds. Generally robust method.
        'xtol':  1e-1, # Tolerance for termination by the change of the independent variables
        'x_scale': [7,7,.5,.5], # Characteristic scale of each variable.
        'bounds': ([-30,-30,-5,-5], [30,30,5,5]),
    }
x0_variables = [4,4,0,0]

In [22]:
lmout = least_squares(
    ss.retrieval.cost_function.fun_residual,
    x0_variables,
    args=(sl1, sn, gmf),
    **opt
)

In [23]:
lmout

 active_mask: array([0, 0, 0, 0])
        cost: 359.28576479202127
         fun: array([-5.84578011,  0.84954006,  3.50292383, 13.55825809,  7.28518438,
        0.        ,  0.        , 20.84480552])
        grad: array([-1.11615664e-01,  1.29430583e+01, -1.06386357e+03, -2.18671219e+03])
         jac: array([[-5.87770001e+00, -3.71199300e+00,  5.87770045e+00,
         3.71199316e+00],
       [-4.73286354e+00, -1.55214514e+00,  4.73286370e+00,
         1.55214574e+00],
       [-3.78176383e+00, -1.51456218e+00,  3.78176421e+00,
         1.51456273e+00],
       [-1.26884807e+00, -1.57279132e-01,  1.26884818e+00,
         1.57279253e-01],
       [ 0.00000000e+00,  0.00000000e+00, -2.22214300e+02,
        -2.22214300e+02],
       [ 0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.66203954e+01,
        -2.66203966e+01]])
     message: '

# Test find_minima

In [24]:
level1

<xarray.Dataset>
Dimensions:              (Antenna: 4, across: 5, along: 6)
Coordinates:
    CentralWavenumber    int64 270
    CentralFreq          float64 1.35e+10
    IncidenceAngleImage  (across, along, Antenna) int64 30 30 30 30 ... 30 30 30
    AntennaAzimuthImage  (across, along, Antenna) int64 45 90 90 ... 90 90 135
    Polarization         (across, along, Antenna) <U2 'VV' 'VV' ... 'HH' 'VV'
  * across               (across) int64 0 1 2 3 4
  * along                (along) int64 0 1 2 3 4 5
  * Antenna              (Antenna) object 'Fore' 'MidV' 'MidH' 'Aft'
Data variables:
    Sigma0               (Antenna, across, along) float64 0.07128 ... 0.1668
    WASV                 (Antenna, across, along) float64 0.3227 ... -1.498
    RVL                  (Antenna, across, along) float64 0.06364 ... -0.5313

In [25]:
L1stack = level1.stack(z=("across", "along"))

In [26]:
noise

<xarray.Dataset>
Dimensions:              (Antenna: 4, across: 5, along: 6)
Coordinates:
    CentralWavenumber    int64 270
    CentralFreq          float64 1.35e+10
    IncidenceAngleImage  (across, along, Antenna) int64 30 30 30 30 ... 30 30 30
    AntennaAzimuthImage  (across, along, Antenna) int64 45 90 90 ... 90 90 135
    Polarization         (across, along, Antenna) <U2 'VV' 'VV' ... 'HH' 'VV'
  * across               (across) int64 0 1 2 3 4
  * along                (along) int64 0 1 2 3 4 5
  * Antenna              (Antenna) object 'Fore' 'MidV' 'MidH' 'Aft'
Data variables:
    Sigma0               (Antenna, across, along) float64 0.003564 ... 0.008339
    RVL                  (Antenna, across, along) float64 0.003182 ... -0.02656

In [27]:
Nstack = noise.stack(z=("across", "along"))

In [28]:
gmf

{'nrcs': {'name': 'nscat4ds'}, 'doppler': {'name': 'mouche12'}}

In [29]:
lmoutmap = [None] * L1stack.z.size

In [31]:
for ii, zindex in enumerate(L1stack.z.data):
    print(ii)
    sL1 = L1stack.sel(z=zindex)
    sN = Nstack.sel(z=zindex)
    lmoutmap[ii] = ss.retrieval.cost_function.find_minima(sL1, sN, gmf)

0
{'x0': array([-9.17862263, -5.72499873, -2.06613787,  1.15869794])}
{'x0': array([-8.9429709 ,  9.39106814, -2.05927423,  2.27858338])}
{'x0': array([ 6.17309597,  9.15541641, -0.93938879,  2.27171974])}
To Be Done find_initial_value
1
{'x0': array([-10.77222641,  -3.88839464,  -1.93481892,   1.63252993])}
{'x0': array([-7.356249  , 11.60068738, -1.83532443,  2.76327989])}
{'x0': array([ 8.13283302,  8.18470997, -0.70457447,  2.6637854 ])}
To Be Done find_initial_value
2
{'x0': array([3.11191024, 8.43650077, 1.7850269 , 0.74323091])}
{'x0': array([ 9.92915935, -1.29438603,  2.23588949,  0.03249731])}
{'x0': array([ 0.19827255, -8.11163515,  1.5251559 , -0.41836528])}
To Be Done find_initial_value
3
{'x0': array([-0.08412185,  8.75052889,  1.38672922,  1.49244616])}
{'x0': array([9.3439296 , 2.52692994, 2.08584688, 1.05607886])}
{'x0': array([ 3.12033065, -6.90112152,  1.64947958,  0.3569612 ])}
To Be Done find_initial_value
4
{'x0': array([ 8.23224152,  5.94015881,  1.77206712, -0.47

In [33]:
lmmap = xr.concat(lmoutmap, dim='z')

In [36]:
lmmap['z'] = L1stack.z

In [39]:
sol = lmmap.unstack(dim='z')

In [48]:
sol

<xarray.Dataset>
Dimensions:            (Ambiguities: 4, Antenna: 4, Observables: 2, across: 5, along: 6, extrema: 2, fun_variables: 8, x_variables: 4)
Coordinates:
  * x_variables        (x_variables) <U3 'u' 'v' 'c_u' 'c_v'
    fun_variables      (Observables, Antenna) int64 0 1 2 3 4 5 6 7
  * Observables        (Observables) <U6 'sigma0' 'RVL'
  * Antenna            (Antenna) object 'Fore' 'MidV' 'MidH' 'Aft'
    CentralWavenumber  int64 270
    CentralFreq        float64 1.35e+10
  * across             (across) int64 0 1 2 3 4
  * along              (along) int64 0 1 2 3 4 5
Dimensions without coordinates: Ambiguities, extrema
Data variables:
    x                  (Ambiguities, x_variables, across, along) float64 5.93...
    active_mask        (Ambiguities, x_variables, across, along) int64 0 0 ... 0
    grad               (Ambiguities, x_variables, across, along) float64 4.24...
    fun                (Ambiguities, Observables, Antenna, across, along) float64 ...
    jac                (Ambiguities, fun_variables, x_variables, across, along) float64 ...
    x0                 (Ambiguities, x_variables, across, along) float64 6.73...
    cost               (Ambiguities, across, along) float64 65.9 ... 1.261e+03
    optimality         (Ambiguities, across, along) float64 1.013e+04 ... 6.9...
    nfev               (Ambiguities, across, along) int64 18 7 31 17 ... 16 31 8
    njev               (Ambiguities, across, along) int64 14 7 30 13 ... 13 26 5
    status             (Ambiguities, across, along) int64 3 3 3 3 3 ... 3 3 3 3
    message            (Ambiguities, across, along) <U42 '`xtol` termination ...
    success            (Ambiguities, across, along) bool True True ... True True
    method             (across, along) <U3 'trf' 'trf' 'trf' ... 'trf' 'trf'
    xtol               (across, along) float64 0.001 0.001 0.001 ... 0.001 0.001
    x_scale            (x_variables, across, along) float64 7.0 7.0 ... 0.5 0.5
    bounds             (extrema, x_variables, across, along) int64 -30 -30 ... 5

In [50]:
sol.isel(along=0, across=0)

<xarray.Dataset>
Dimensions:            (Ambiguities: 4, Antenna: 4, Observables: 2, extrema: 2, fun_variables: 8, x_variables: 4)
Coordinates:
  * x_variables        (x_variables) <U3 'u' 'v' 'c_u' 'c_v'
    fun_variables      (Observables, Antenna) int64 0 1 2 3 4 5 6 7
  * Observables        (Observables) <U6 'sigma0' 'RVL'
  * Antenna            (Antenna) object 'Fore' 'MidV' 'MidH' 'Aft'
    CentralWavenumber  int64 270
    CentralFreq        float64 1.35e+10
    across             int64 0
    along              int64 0
Dimensions without coordinates: Ambiguities, extrema
Data variables:
    x                  (Ambiguities, x_variables) float64 5.937 ... 0.5925
    active_mask        (Ambiguities, x_variables) int64 0 0 0 0 0 ... 0 0 0 0 0
    grad               (Ambiguities, x_variables) float64 4.248 ... -2.392e+03
    fun                (Ambiguities, Observables, Antenna) float64 2.113 ... ...
    jac                (Ambiguities, fun_variables, x_variables) float64 -1.8...
    x0                 (Ambiguities, x_variables) float64 6.735 ... 2.272
    cost               (Ambiguities) float64 65.9 46.16 6.206e-05 96.65
    optimality         (Ambiguities) float64 1.013e+04 1.372e+03 113.7 1.462e+06
    nfev               (Ambiguities) int64 18 29 12 13
    njev               (Ambiguities) int64 14 24 11 7
    status             (Ambiguities) int64 3 3 3 3
    message            (Ambiguities) <U42 '`xtol` termination condition is sa...
    success            (Ambiguities) bool True True True True
    method             <U3 'trf'
    xtol               float64 0.001
    x_scale            (x_variables) float64 7.0 7.0 0.5 0.5
    bounds             (extrema, x_variables) int64 -30 -30 -5 -5 30 30 5 5

In [49]:
sol.isel(along=0, across=0).sortby('cost')

<xarray.Dataset>
Dimensions:            (Ambiguities: 4, Antenna: 4, Observables: 2, extrema: 2, fun_variables: 8, x_variables: 4)
Coordinates:
  * x_variables        (x_variables) <U3 'u' 'v' 'c_u' 'c_v'
    fun_variables      (Observables, Antenna) int64 0 1 2 3 4 5 6 7
  * Observables        (Observables) <U6 'sigma0' 'RVL'
  * Antenna            (Antenna) object 'Fore' 'MidV' 'MidH' 'Aft'
    CentralWavenumber  int64 270
    CentralFreq        float64 1.35e+10
    across             int64 0
    along              int64 0
Dimensions without coordinates: Ambiguities, extrema
Data variables:
    x                  (Ambiguities, x_variables) float64 -4.503 7.8 ... 0.5925
    active_mask        (Ambiguities, x_variables) int64 0 0 0 0 0 ... 0 0 0 0 0
    grad               (Ambiguities, x_variables) float64 1.693e-08 ... -2.39...
    fun                (Ambiguities, Observables, Antenna) float64 0.001135 ....
    jac                (Ambiguities, fun_variables, x_variables) float64 0.35...
    x0                 (Ambiguities, x_variables) float64 -8.943 9.391 ... 2.272
    cost               (Ambiguities) float64 6.206e-05 46.16 65.9 96.65
    optimality         (Ambiguities) float64 113.7 1.372e+03 1.013e+04 1.462e+06
    nfev               (Ambiguities) int64 12 29 18 13
    njev               (Ambiguities) int64 11 24 14 7
    status             (Ambiguities) int64 3 3 3 3
    message            (Ambiguities) <U42 '`xtol` termination condition is sa...
    success            (Ambiguities) bool True True True True
    method             <U3 'trf'
    xtol               float64 0.001
    x_scale            (x_variables) float64 7.0 7.0 0.5 0.5
    bounds             (extrema, x_variables) int64 -30 -30 -5 -5 30 30 5 5